In [1]:
import pandas as pd
from functools import partial, reduce

In [2]:
# Behaviors per Mechanism
def b1m1(step, sL, s):
    return s['s1'] + 1
def b2m1(step, sL, s):
    return s['s1'] + 1

def b1m2(step, sL, s):
    return s['s1'] + 1
def b2m2(step, sL, s):
    return s['s1'] + 1

def b1m3(step, sL, s):
    return s['s1'] + 1
def b2m3(step, sL, s):
    return s['s2'] + 1


# Internal States per Mechanism
def s1m1(step, sL, s, _input):
    y = 's1'
    x = s['s1'] + _input
    return (y, x)
def s2m1(step, sL, s, _input):
    y = 's2'
    x = s['s2'] + _input
    return (y, x)

def s1m2(step, sL, s, _input):
    y = 's1'
    x =  s['s1'] + _input
    return (y, x)
def s2m2(step, sL, s, _input):
    y = 's2'
    x =  s['s2'] + _input
    return (y, x)

def s1m3(step, sL, s, _input):
    y = 's1'
    x = s['s1'] + _input
    return (y, x)
def s2m3(step, sL, s, _input):
    y = 's2'
    x =  s['s2'] + _input
    return (y, x)

# Exogenous States
proc_one_coef_A = 0.7
proc_one_coef_B = 1.3
def es3p1(step, sL, s, _input):
    y = 's3'
    x = s['s3'] * bound_norm_random(seed['a'], proc_one_coef_A, proc_one_coef_B)
    return (y, x)
def es4p2(step, sL, s, _input):
    y = 's4'
    x = s['s4'] * bound_norm_random(seed['b'], proc_one_coef_A, proc_one_coef_B)
    return (y, x)
def es5p2(step, sL, s, _input): # accept timedelta instead of timedelta params
    y = 'timestamp'
    x = ep_time_step(s, s['timestamp'], seconds=1)
    return (y, x)



# Environment States
def env_a(x):
    return 10
def env_b(x):
    return 10


exogenous_states = {
    "s3": es3p1,
    "s4": es4p2,
    "timestamp": es5p2
}

ep = list(exogenous_states.values())

mechanisms = {
    "m1": {
        "behaviors": {
            "b1": b1m1, # lambda step, sL, s: s['s1'] + 1,
            "b2": b2m1
        },
        "states": {
            "s1": s1m1,
            "s2": s2m1,
        }
    },
    "m2": {
        "behaviors": {
            "b1": b1m2,
            "b2": b2m2
        },
        "states": {
            "s1": s1m2,
            # "s2": s2m2,
        }
    },
    "m3": {
        "behaviors": {
            # "b1": b1m3,
            # "b2": b2m3
        },
        "states": {
            "s1": s1m3,
            # "s2": s2m3,
        }
    }
}


def state_identity(k):
    def identity(step, sL, s, _input):
        return (k, s[k])
    return identity

def behavior_identity(k):
    def identity(step, sL, s):
        return s[k]
    return identity

def key_filter(mechanisms, keyname):    
    return [ v[keyname] for k, v in mechanisms.items() ]

def fillna_with_id(identity, df, col):
    return df[[col]].fillna(value=identity(col))

def apply_identity_funcs(identity, df, cols):
    return list(map(lambda col: fillna_with_id(identity, df, col), cols))

def create_matrix_field(mechanisms, key):
    if key == 'states':
        identity = state_identity
    else:
        identity = behavior_identity
    df = pd.DataFrame(key_filter(mechanisms, key))
    col_list = apply_identity_funcs(identity, df, list(df.columns))
    return reduce((lambda x, y: pd.concat([x, y], axis=1)), col_list)

def create_tensor_field(mechanisms, env_poc=ep, keys=['behaviors', 'states']):
    dfs = [ create_matrix_field(mechanisms, k) for k in keys ]
    df = pd.concat(dfs, axis=1)
    for es, i in zip(env_poc, range(len(env_poc))):
        df['es'+str(i)] = es
    df['m'] = df.index + 1
    return df

def generate_config(mechanisms, env_poc=ep):
    bdf = create_matrix_field(mechanisms,'behaviors')
    sdf = create_matrix_field(mechanisms,'states')
    zipped_list = list(zip(sdf.values.tolist(), bdf.values.tolist()))
    return list(map(lambda x: (x[0] + env_poc, x[1]), zipped_list))

In [3]:
create_matrix_field(mechanisms,'behaviors').values.tolist()
        
# exdf[exdf.index == 0].values.tolist() #['b1','b2']

[[<function __main__.b1m1(step, sL, s)>,
  <function __main__.b2m1(step, sL, s)>],
 [<function __main__.b1m2(step, sL, s)>,
  <function __main__.b2m2(step, sL, s)>],
 [<function __main__.behavior_identity.<locals>.identity(step, sL, s)>,
  <function __main__.behavior_identity.<locals>.identity(step, sL, s)>]]

In [4]:
# generate_config(mechanisms, ep)
create_tensor_field(mechanisms)

,b1,b2,s1,s2,es0,es1,es2,m
0,<function b1m1 at 0x10af42158>,<function b2m1 at 0x10af42268>,<function s1m1 at 0x10af42510>,<function s2m1 at 0x10af42598>,<function es3p1 at 0x10af42840>,<function es4p2 at 0x10af428c8>,<function es5p2 at 0x10af42950>,1
1,<function b1m2 at 0x10af422f0>,<function b2m2 at 0x10af42400>,<function s1m2 at 0x10af42620>,<function state_identity.<locals>.identity at ...,<function es3p1 at 0x10af42840>,<function es4p2 at 0x10af428c8>,<function es5p2 at 0x10af42950>,2
2,<function behavior_identity.<locals>.identity ...,<function behavior_identity.<locals>.identity ...,<function s1m3 at 0x10af42730>,<function state_identity.<locals>.identity at ...,<function es3p1 at 0x10af42840>,<function es4p2 at 0x10af428c8>,<function es5p2 at 0x10af42950>,3


In [5]:
# def generate_config(mechanisms, exogenous_states):
#     es_funcs = list(exogenous_states.values())
#     # es_funcs = [ exogenous_states[state] for state in list(exogenous_states.keys()) ]
#     config = list(
#         map(
#             lambda m: (
#                 list(mechanisms[m]["states"].values()) + es_funcs,
#                 list(mechanisms[m]["behaviors"].values())
#             ),
#             list(mechanisms.keys())
#         )
#     )
#     return config
# generate_config(mechanisms, exogenous_states)

[([<function __main__.s1m1(step, sL, s, _input)>,
   <function __main__.s2m1(step, sL, s, _input)>,
   <function __main__.es3p1(step, sL, s, _input)>,
   <function __main__.es4p2(step, sL, s, _input)>,
   <function __main__.es5p2(step, sL, s, _input)>],
  [<function __main__.b1m1(step, sL, s)>,
   <function __main__.b2m1(step, sL, s)>]),
 ([<function __main__.s1m2(step, sL, s, _input)>,
   <function __main__.es3p1(step, sL, s, _input)>,
   <function __main__.es4p2(step, sL, s, _input)>,
   <function __main__.es5p2(step, sL, s, _input)>],
  [<function __main__.b1m2(step, sL, s)>,
   <function __main__.b2m2(step, sL, s)>]),
 ([<function __main__.s1m3(step, sL, s, _input)>,
   <function __main__.es3p1(step, sL, s, _input)>,
   <function __main__.es4p2(step, sL, s, _input)>,
   <function __main__.es5p2(step, sL, s, _input)>],
  [])]

In [170]:
create_tensor_field(mechanisms)

,b1,b2,s1,s2,es0,es1,es2,m
0,<function b1m1 at 0x118771e18>,<function b2m1 at 0x1188d7d90>,<function s1m1 at 0x1188d7730>,<function s2m1 at 0x1188ed730>,<function es3p1 at 0x1188ed400>,<function es4p2 at 0x1188ed378>,<function es5p2 at 0x1188ed158>,1
1,<function b1m2 at 0x1188d70d0>,<function b2m2 at 0x1188d7840>,<function s1m2 at 0x1188ed620>,<function state_identity.<locals>.identity at ...,<function es3p1 at 0x1188ed400>,<function es4p2 at 0x1188ed378>,<function es5p2 at 0x1188ed158>,2
2,<function b1m3 at 0x1188d7ea0>,<function behavior_identity.<locals>.identity ...,<function s1m3 at 0x1188d7a60>,<function s2m3 at 0x1188ed268>,<function es3p1 at 0x1188ed400>,<function es4p2 at 0x1188ed378>,<function es5p2 at 0x1188ed158>,3


In [53]:
create_matrix_field(mechanisms,'behaviors')
create_matrix_field(mechanisms,'states')

,b1,b2
0,<function b1m1 at 0x1108c88c8>,<function b2m1 at 0x1108c89d8>
1,<function b1m2 at 0x1108c8d90>,<function b2m2 at 0x1108c8a60>
2,<function b1m3 at 0x1108c8c80>,<function behavior_identity.<locals>.identity ...


In [60]:
tensor_field_report(mechanisms)

,b1,b2,s1,s2,es0,es1,es2,m
0,<function b1m1 at 0x1105ed9d8>,<function b2m1 at 0x1105edbf8>,<function s1m1 at 0x1105c69d8>,<function s2m1 at 0x1105ede18>,<function es3p1 at 0x1187710d0>,<function es4p2 at 0x118771158>,<function es5p2 at 0x1187711e0>,1
1,<function b1m2 at 0x1105edc80>,<function b2m2 at 0x1105edd08>,<function s1m2 at 0x1105edea0>,<function state_identity.<locals>.identity at ...,<function es3p1 at 0x1187710d0>,<function es4p2 at 0x118771158>,<function es5p2 at 0x1187711e0>,2
2,<function b1m3 at 0x1105edb70>,<function behavior_identity.<locals>.identity ...,<function s1m3 at 0x1105ed0d0>,<function s2m3 at 0x118771048>,<function es3p1 at 0x1187710d0>,<function es4p2 at 0x118771158>,<function es5p2 at 0x1187711e0>,3


In [39]:

for x, i in zip(['a', 'b', 'c'], range(3)):
    print((x, i))

('a', 0)
('b', 1)
('c', 2)


In [41]:
len(env_poc)

3

In [ ]:
def create_tensor_field2(mechanisms, env_poc=ep):
    beh_df = create_matrix_field(mechanisms, 'behaviors')
    state_df = create_matrix_field(mechanisms, 'states')
    ep_df = pd.DataFrame({'m' : range(len(env_poc))})
    for es, i in zip(env_poc, range(len(env_poc))):
        ep_df['es'+str(i)] = es
        
    return beh_df